In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tools import CN
import matplotlib.pylab as plt
from diffrl.env import DifferentiablePhysicsEnv

# 1.19

env = DifferentiablePhysicsEnv.parse(
    cfg_path='configs/plb_cuda.yml',
    sim_cfg=dict(yield_stress=1000.),

    #cfg_path='configs/plb3d.yml',
    #sim_cfg=dict(quality=1.0, yield_stress=1000.),
    #use_taichi=True,

    task_cfg=dict(TYPE="MoveObject", shape_metric=dict(chamfer=1., center=10.)),
    solver=dict(
        render_path=None,
        render_interval=50,
        max_iter=1, lr=0.1
    ),
    **{"manipulator_cfg.size": (0.05, 0.2, 0.2)},
)

Session:
  signature_scheme="hmac-sha256": --Session.key=b"6c0c9b99-5aed-4a60-a55a-4dfafea7ab08"
control=9001: --hb=9000
iopub=9004: --f=/tmp/tmp-13267d4RL2WC1z83y.json
ip=127:
  0:
    0:
      1: --stdin=9003
shell=9002: --transport="tcp" 
 ------------------------------------------------------------ 
 cfg_path: configs/plb_cuda.yml
device: cuda:0
manipulator_cfg:
  CLASS: Manipulator
  TYPE: ParallelGripper
  size: (0.05, 0.2, 0.2)
max_episode_steps: 50
observer_cfg:
  CLASS: Observer
  TYPE: TaichiRGBDObserver
sim_cfg:
  yield_stress: 1000.0
solver:
  compute_loss_in_end: False
  early_stop: None
  lr: 0.1
  max_iter: 1
  optim_type: Adam
  render_interval: 50
  render_path: None
  save_state: 1
  verbose: True
task_cfg:
  CLASS: Task
  TYPE: MoveObject
  shape_metric:
    center: 10.0
    chamfer: 1.0
use_taichi: False
warning_off: True
ERROR while adding argumetns before!!! 'Non-existent config key: control=9001'
E: 5000.0
gravity: (0.0, -1.0, 0.0)
grid_size: (1.0, 1.0, 1.0)
grou

In [3]:
# out = env.solve(env.task.reset(env), [env.manipulator.get_initial_action() for i in range(50)], max_iter=30)

In [4]:

# env.execute(out['initial_state'], out['best_action'], 'animation.mp4')
import pickle
#with open('test_data.pkl', 'wb') as f:
#    pickle.dump(out, f)
import pickle
    
with open('test_data.pkl', 'rb') as f:
    out = pickle.load(f)

    


In [5]:

import torch
actions = torch.nn.Parameter(torch.tensor(out['best_action'], dtype=torch.float32, device='cuda:0'))
env.set_state(out['initial_state'])

sim = env.simulator
func = env.torch_env

T = 50
sim.substeps = 2


func.zero_grad()
obs = func.get_obs(0, device='cuda:0')
loss = 0
for idx, a in enumerate(actions[:T]):
    obs = func.forward(idx, a, *obs)
    #loss = loss + obs[0][:, -2:].mean()
    #loss = loss + obs[0][:, -2:].mean() + ((obs[0][:, :3] - torch.tensor([0.2, 0.5, 0.4], device='cuda:0'))**2).mean()
    if idx == T-1:
        loss = loss + ((obs[0][:, :3] - torch.tensor([0.2, 0.5, 0.4], device='cuda:0'))**2).mean()

    #print(((obs[0][:, :3] - torch.tensor([0.2, 0.5, 0.4], device='cuda:0'))**2).mean())

#for i in range(100):
    #print('x grad', sim.states[i].x.download().mean(axis=0), 'v_grad', sim.states[i].v.download().mean(axis=0), 'pos', sim.states[i].body_pos_grad.download().sum())

print('loss', loss)

loss.backward()
print(actions.shape)
print(actions.grad[:T])

loss tensor(0.0839, device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([50, 12])
tensor([[ 5.6185e-05, -1.9683e-04, -3.0821e-05, -5.5079e-07,  1.2292e-04,
         -3.1712e-04,  8.9590e-03,  9.8980e-04, -2.2514e-04,  7.3691e-07,
         -1.2303e-06,  6.5608e-04],
        [ 4.1015e-05, -2.3320e-04, -4.5915e-05,  1.7021e-06,  1.2198e-04,
         -3.2067e-04,  9.0671e-03,  9.3638e-04, -2.0060e-04,  2.3194e-06,
          2.4262e-06,  6.5723e-04],
        [ 2.6113e-04, -7.1873e-04, -3.2294e-05,  7.6684e-06,  1.2200e-04,
         -3.5323e-04,  9.0461e-03,  6.8473e-04, -8.8292e-05,  3.2677e-06,
          9.5949e-06,  6.7343e-04],
        [ 3.5987e-04, -4.5480e-04, -8.4854e-05,  6.2795e-06,  1.2386e-04,
         -3.3053e-04,  9.0753e-03,  6.8018e-04, -9.4724e-05,  3.3725e-06,
          6.5410e-06,  6.7173e-04],
        [ 4.4843e-04, -1.9150e-03,  3.4973e-05,  2.1049e-05,  1.2100e-04,
         -4.2698e-04,  9.1419e-03,  7.6506e-04, -1.3789e-04,  4.4202e-06,
         -9.4602e-07,  6.6426e-04]

In [6]:
print(sim.n_particles)
for i in range(sim.substeps*T+1):
    #print(i, 'pos', sim.states[i].body_pos_grad.download().mean())
    print(i, sim.states[i].x_grad.download()[:625].mean(), sim.states[i].v_grad.download()[:625].mean())
    #print(i, sim.states[i].F_grad.download()[:625].mean(), sim.states[i].v_grad.download()[:625].mean())

625
0 -0.00014839307 -2.3316566e-07
1 -0.00014839375 -2.1878807e-07
2 -0.00014344126 -2.0501615e-07
3 -0.00012747703 -1.9275258e-07
4 -0.00012583584 -1.8071796e-07
5 -0.00012224332 -1.697414e-07
6 -0.00014731614 -1.6061398e-07
7 -0.00013207228 -1.494948e-07
8 -0.00012351606 -1.3734511e-07
9 -0.00014134735 -1.3257687e-07
10 -0.00014319642 -1.2321735e-07
11 -0.00014022706 -1.0859083e-07
12 -0.00017332564 -1.08651975e-07
13 -0.00019125678 -1.0575092e-07
14 -0.00020356146 -9.4071474e-08
15 -0.00016573994 -8.2046e-08
16 -0.00016424716 -6.856347e-08
17 -0.00016833213 -5.9182724e-08
18 -0.000169515 -6.762879e-08
19 -0.00017533897 -5.2180802e-08
20 -0.00018029651 -2.9768554e-08
21 -0.00020544458 -1.550692e-08
22 -0.00022510729 -1.8073662e-08
23 -0.00021596835 -1.0061532e-08
24 -0.00022099933 -2.2471964e-09
25 -0.00020954083 -1.960255e-09
26 -0.00019276212 -5.8197545e-09
27 -0.0001419908 -4.109084e-09
28 -0.000104650804 -5.8669345e-09
29 -5.949444e-05 -2.662765e-10
30 -6.885743e-06 3.3975493e-1

In [7]:
sim.states[1].clear_grad(sim.stream0)
sim.substep_grad(1)
print(sim.body_type_mu_softness_round.download())
print(sim.states[1].v_grad.download(sim.n_particles).mean())
print(sim.temp.grid_v_in_grad.download().mean())
print(sim.temp.grid_v_out_grad.download().mean())
#print(sim.states[2].body_pos.download())
#print('pos', sim.states[1].body_pos_grad.download())
#print(sim.states[2].body_rot_grad.download())

[[0.00e+00 5.00e+00 6.66e+02 1.00e-02]
 [0.00e+00 5.00e+00 6.66e+02 1.00e-02]]
-2.1878806e-07
-9.279194e-06
-5.229957e-10


In [8]:
xx = pickle.load(open('xx.pkl', 'rb'))
import numpy as np

v = np.abs(sim.temp.grid_v_in_grad.download().reshape(64, 64, 64, 3) - xx).argmax()
v = v//3
z = v % 64; v//=64
y = v % 64; v//=64
x = v % 64; v//=64
print(x, y, z, v)

print(sim.temp.grid_body_v_in.download().reshape(64, 64, 64, 3, 3)[x, y, z])
print(sim.temp.grid_v_in_grad.download().reshape(64, 64, 64, 3)[x, y, z])
print('vout', sim.temp.grid_v_out.download().reshape(64, 64, 64, 3)[x, y, z])
print('ground friction', sim.get_ground_friction())

print(xx[x, y, z])

36 11 35 0
[[ 1.934833e-05 -5.991046e-03 -4.134950e-06]
 [ 1.934833e-05 -5.991046e-03 -4.134950e-06]
 [ 1.934833e-05 -5.991046e-03 -4.134950e-06]]
[-0.02264165 -0.02054178  0.01326743]
vout [ 1.934833e-05 -5.991046e-03 -4.134950e-06]
ground friction 5.0
[ 0.00547015 -0.01412182  0.00534411]


In [9]:
from utils import set_mpm_grad, compare_grid, compare_p

with open('substep_4.pkl', 'rb') as f:
    data = pickle.load(f)

set_mpm_grad(sim, 1, data[1])
set_mpm_grad(sim, 0, data[0])

sim.states[0].clear_grad(sim.stream0)
sim.temp.clear(sim.stream0)
sim.temp.clear_grad(sim.stream0)
n = sim.n_particles

sim.substep_grad(0)

svd = data[-1][-1]
svd_grad = data[-1][-2]
"""
cur = sim.states[0]
next = sim.states[1]
temp = sim.temp
sim.compute_svd(cur, temp)
sim.temp.grid_m.upload(data[2][4].reshape(-1))
sim.temp.grid_m_grad.upload(data[2][5].reshape(-1))
sim.temp.grid_v_in.upload(data[2][0].reshape(-1, 3))
sim.temp.grid_v_in_grad.upload(data[2][1].reshape(-1, 3))

sim.temp.grid_v_out.upload(data[2][2].reshape(-1, 3))
# sim.temp.grid_v_out_grad.upload(data[2][3].reshape(-1, 3))

# sim.U.to_numpy(), sim.V.to_numpy(), sim.sig.to_numpy(), sim.F_tmp.to_numpy()
sim.temp.U.upload(svd[0].reshape(-1, 9))
sim.temp.V.upload(svd[1].reshape(-1, 9))
sim.temp.sig.upload(svd[2].reshape(-1, 3, 3)[:, [0, 1, 2], [0, 1, 2]])
sim.temp.F.upload(svd[3].reshape(-1, 9))

sim.g2p_grad(cur, temp, next)
sim.p2g_grad(cur, temp, next)

# compare_p(sim.temp.F_grad.download(n), data[-1][-3], 'sig_grad_before')

sim.temp.U_grad.upload(svd_grad[0].reshape(-1, 9))
sim.temp.V_grad.upload(svd_grad[1].reshape(-1, 9))
sim.temp.sig_grad.upload(svd_grad[2].reshape(-1, 3, 3)[:, [0, 1, 2], [0, 1, 2]])

sim.compute_svd_grad(cur, temp)
"""


#print(np.abs(sim.temp.grid_v_out.download().reshape(64, 64, 64, 3)-data[2][2]).max())
#print(np.abs(sim.temp.grid_v_out_grad.download().reshape(64, 64, 64, 3)-data[2][3]).max())

print('grid_v_out')
compare_grid(sim.temp.grid_v_out.download(), data[2][2])
# compare_grid(sim.temp.grid_v_in.download(), data[2][0])
# compare_grid(sim.temp.grid_v_in_grad.download(), data[2][1])

# compare_p(sim.temp.U_grad.download(n), svd_grad[0], 'U_grad')
# compare_p(sim.temp.V_grad.download(n), svd_grad[1], 'V_grad')
#print('402 sig', sim.temp.sig.download(n)[402], svd[2][402])
#print('402 sig', sim.temp.sig_grad.download(n)[402], svd_grad[2][402])
#compare_p(sim.temp.sig_grad.download(n), svd_grad[2][:, [0,1,2], [0, 1, 2]], 'sig_grad')
compare_p(sim.temp.F_grad.download(n), svd_grad[3], 'F_grad')
print('grid v in')
compare_grid(sim.temp.grid_v_in_grad.download(), data[2][1])
# print('mean', data[0]['grad'][0].mean())
# print(sim.states[0].x_grad.download(n).mean())
#compare_p(sim.states[0].x_grad.download(n), data[0]['grad'][0], 'x')
#compare_p(sim.states[0].v_grad.download(n), data[0]['grad'][1], 'v')
#compare_p(sim.states[0].F_grad.download(n), data[0]['grad'][2], 'F')
#compare_p(sim.states[0].C_grad.download(n), data[0]['grad'][3], 'C')



print(np.array(data[0]['grad'][4:])[:, :3])
print('old next', np.array(data[1]['grad'][4:])[:, :3]) # futher + gridop

print(sim.states[0].body_pos_grad.download())
print('next', sim.states[1].body_pos_grad.download() - np.array(data[1]['grad'][4:])[:, :3])
print(sim.states[0].body_pos_grad.download() + np.array(data[1]['grad'][4:])[:, :3])
compare_p(sim.states[0].body_pos_grad.download() + np.array(data[1]['grad'][4:])[:, :3],
          np.array(data[0]['grad'][4:])[:, :3], 'pos')

# compare_p(sim.states[0].body_rot_grad.download() + sim.states[1].body_pos_grad.download(), np.array(data[0]['grad'][4:])[:, 3:], 'rot')


#print('v1', sim.temp.grid_v_in.download().reshape(64, 64, 64, 3)[34, 2, 29])
#print('v2', sim.temp.grid_v_out.download().reshape(64, 64, 64, 3)[34, 2, 29])
#print(data[2][0][34, 2, 29])
#print(data[2][2][34, 2, 29])
#print(sim.temp.grid_body_v_in.download().reshape(64, 64, 64, 3, 3)[34, 2, 29])

grid_v_out
33 2 36 [0.16404751 0.         0.03439872] [0.16419101 0.         0.03443293] 0.00014349818
F_grad 577 [[ 3.9860606e-07  4.2431056e-06 -5.3197145e-06]
 [ 4.2468309e-06 -1.1920929e-05 -1.1563301e-05]
 [-5.3159893e-06 -1.1533499e-05  1.7732382e-05]] [[ 3.99454734e-07  4.24348491e-06 -5.32089143e-06]
 [ 4.26158931e-06 -1.20085988e-05 -1.16152405e-05]
 [-5.31708292e-06 -1.15217454e-05  1.77274724e-05]] 8.766983e-08
grid v in
33 2 36 [-0.01658057 -0.08346934 -0.00227173] [-0.01658054 -0.08346827 -0.00227101] 1.065433e-06
[[ 0.02040918 -0.03078845  0.02363042]
 [-0.16228034 -0.00314798 -0.00459299]]
old next [[ 0.02440064 -0.03013135  0.02272516]
 [-0.16228034 -0.00314798 -0.00459299]]
[[ 0.02729355  0.02924888 -0.01842774]
 [ 0.          0.          0.        ]]
next [[-0.02730482 -0.02924369  0.01841929]
 [ 0.          0.          0.        ]]
[[ 0.0516942  -0.00088247  0.00429743]
 [-0.16228034 -0.00314798 -0.00459299]]
pos 0 [ 0.0516942  -0.00088247  0.00429743] [ 0.02040918 -

In [10]:
sim.temp.clear_grad(sim.stream0)
set_mpm_grad(sim, 1, data[1])
set_mpm_grad(sim, 0, data[0])
sim.states[0].clear_grad(sim.stream0)
sim.states[1].clear_grad(sim.stream0)

sim.temp.grid_m.upload(data[2][4].reshape(-1))
sim.temp.grid_v_in.upload(data[2][0].reshape(-1, 3))
sim.temp.grid_v_out_grad.upload(data[2][3].reshape(-1, 3))

cur, temp, next = sim.states[0], sim.temp, sim.states[1]
sim.grid_op(cur, temp, next)
sim.grid_op_grad(cur, temp, next)

print(sim.states[0].body_pos_grad.download())
print(sim.states[1].body_pos_grad.download())
#print(sim.states[1].body_pos_grad.download() - np.array(data[1]['grad'][4:])[:, :3])
print(sim.states[0].body_rot_grad.download())
print(sim.states[1].body_rot_grad.download())
#print(sim.states[1].body_rot_grad.download() - np.array(data[1]['grad'][4:])[:, 3:])
#print(sim.states[1].body_pos_grad.download())

[[ 0.02729355  0.02924889 -0.01842773]
 [ 0.          0.          0.        ]]
[[-0.02730483 -0.02924368  0.01841928]
 [ 0.          0.          0.        ]]
[[-5.9815839e-06 -1.2637789e-03  2.4523719e-03  2.3323635e-03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]
[[ 6.4455285e-06  1.2637150e-03 -2.4524282e-03 -2.3323437e-03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]


In [11]:
from utils import backward_svd


backward_svd(svd_grad[0][190], svd_grad[2][190], svd_grad[1][190], svd[0][190], svd[2][190], svd[1][190])  + data[-1][-3][190]

array([[-2.6564990e-06,  2.3636348e-06,  4.5951351e-06],
       [ 2.3647240e-06,  1.3534265e-05, -5.8740893e-06],
       [ 4.5887118e-06, -5.8752812e-06,  8.2864602e-07]], dtype=float32)

In [12]:
sim.torch_scale

tensor([[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100]], device='cuda:0')